# 제주도 핫플레이스 - 인스타그램 크롤링

In [28]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [29]:
import unicodedata
import re
from tqdm.notebook import tqdm

In [30]:
chromedriver = 'C:\workspace\chromedriver\chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

## 1. Instagram에 접속하여 로그인

In [33]:
insta_url = 'https://www.instagram.com'
driver.get(insta_url)
time.sleep(1)

In [35]:
with open('password.txt') as f:
    password = f.read()

In [36]:
email = 'dlgkstoa123@paran.com'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [37]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [38]:
# 로그인 정보 저장 - 나중에 하기
try:
    driver.find_element_by_css_selector('.sqd0P.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [40]:
# 알림 설정 - 나중에 하기
try:
    driver.find_element_by_css_selector('.a00lW.HoLwm').click()
    time.sleep(1)
except:
    pass

## 2. 제주도맛집으로 검색

In [41]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91'

In [42]:
driver.get(url)
time.sleep(5)

## 3. 첫번째 게시글 열기

In [43]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

## 4. 게시글 정보 가져오기

In [44]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [45]:
# 1) 본문 내용 가져오기

try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ' '
content

'⠀<별맛해장국>⠀-제주도 남쪽 끝 남원읍 조용한 시골마을에위치한 별맛해장국 간판만 봐도 맛집 스멜이 느껴지는데~🤗⠀-내장탕이나 나른 메뉴는 없고 오로지소고기선지해장국 하나만 있다⠀-이런 메뉴판은 무조건 믿고 먹는 집이지선지 상태 좋고 부드럽게 씹히는 넉넉한소고기에 구수하고 담백한 국물까지ㅎㅎ⠀-해장하러 왔다 소주시킬 뻔한 걸 간심히참고 한 뚝배기 완뚝~😝⠀-조용한 마을에서 느껴지는 편안함과 맛집느낌의 가게 분위기까지 여기 내 스타일인데ㅋ⠀-가격도 8,000원에 주류 가격도 좋아서귀포쪽에 있다면 한번 들려볼 만한 해장국집이다~😋⠀-아주머님의 정이 느껴지는 별맛해장국소고기해장국 한그릇 잘 먹었습니다~😋..........#별맛해장국#제주도맛집#서귀포맛집#남원맛집#제주도해장국#서귀포해장국#해장국맛집#제주해장국맛집#소고기해장국맛집#선지해장국맛집'

In [46]:
# 2) 본문 내용에서 해쉬태그 가져오기, '#[^\s#,\\]+'
tags = re.findall(r'#[^\s#,\\]+', content)
tags

['#별맛해장국',
 '#제주도맛집',
 '#서귀포맛집',
 '#남원맛집',
 '#제주도해장국',
 '#서귀포해장국',
 '#해장국맛집',
 '#제주해장국맛집',
 '#소고기해장국맛집',
 '#선지해장국맛집']

In [47]:
# 3) 작성일자 정보 가져오기
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

'2021-08-24'

In [48]:
# 4) 좋아요 수 가져오기: 좋아요
try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like = 0
like

'1,058'

In [51]:
# 5) 위치정보 가져오기
try:
    place = soup.select('div.M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

'별맛해장국'

In [59]:
row = [content, date, like, place, tags]
row

['⠀<별맛해장국>⠀-제주도 남쪽 끝 남원읍 조용한 시골마을에위치한 별맛해장국 간판만 봐도 맛집 스멜이 느껴지는데~🤗⠀-내장탕이나 나른 메뉴는 없고 오로지소고기선지해장국 하나만 있다⠀-이런 메뉴판은 무조건 믿고 먹는 집이지선지 상태 좋고 부드럽게 씹히는 넉넉한소고기에 구수하고 담백한 국물까지ㅎㅎ⠀-해장하러 왔다 소주시킬 뻔한 걸 간심히참고 한 뚝배기 완뚝~😝⠀-조용한 마을에서 느껴지는 편안함과 맛집느낌의 가게 분위기까지 여기 내 스타일인데ㅋ⠀-가격도 8,000원에 주류 가격도 좋아서귀포쪽에 있다면 한번 들려볼 만한 해장국집이다~😋⠀-아주머님의 정이 느껴지는 별맛해장국소고기해장국 한그릇 잘 먹었습니다~😋..........#별맛해장국#제주도맛집#서귀포맛집#남원맛집#제주도해장국#서귀포해장국#해장국맛집#제주해장국맛집#소고기해장국맛집#선지해장국맛집',
 '2021-08-24',
 '1,058',
 '별맛해장국',
 ['#별맛해장국',
  '#제주도맛집',
  '#서귀포맛집',
  '#남원맛집',
  '#제주도해장국',
  '#서귀포해장국',
  '#해장국맛집',
  '#제주해장국맛집',
  '#소고기해장국맛집',
  '#선지해장국맛집']]

## 5. 다음 게시글로 이동

In [71]:
driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
time.sleep(2)

In [62]:
# 6을 하기 위해 popup 화면을 없앰(즉, x를 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

## 6. 여러개의 게시글 가져오기

In [69]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]

    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, date, like, place, tags]

## -제주도맛집, 제주맛집, 제주도관광, 제주여행

In [73]:
keyword = '제주도관광'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(1)

In [74]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [75]:
# 200개 정도가 한계인 것 같음
results = []
for _ in tqdm(range(200)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(3)
    except:
        time.sleep(3)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(3)

  0%|          | 0/200 [00:00<?, ?it/s]

In [76]:
df = pd.DataFrame(results, columns=['content','date','like','place','tags'])
df.head()

,content,date,like,place,tags
0,#제주#제주특별시#제주도#제주도관광#성산읍#백패킹#섬패킹#등산#산스타그램🌳#제주백패...,2020-07-24,565,,"[#제주, #제주특별시, #제주도, #제주도관광, #성산읍, #백패킹, #섬패킹, ..."
1,#제주#제주특별시#제주도#제주도관광#백패킹#섬패킹#등산#산스타그램🌳#제주백패킹#백패...,2020-12-09,556,,"[#제주, #제주특별시, #제주도, #제주도관광, #백패킹, #섬패킹, #등산, #..."
2,..늦잠 자고 싶다.6개월 늦은 사진..여기도 제주다.#제주도사진찍기좋은곳 #제주도...,2021-08-23,112,,"[#제주도사진찍기좋은곳, #제주도풍경, #제주도여행, #제주도카페, #제주도숙소, ..."
3,*자구내포구(차귀도포구) / 포구차귀도 배낚시가 유명해서 배낚시를 하러 이 곳 포구...,2021-08-21,"1,260",자구내포구,"[#자구내포구, #차귀도포구, #고산포구, #차귀도, #제주일몰, #제주바다, #제..."
4,💛💛💛10월의 제주도는 억새의 계절 ! 🌾사람도 없구 너무 한적하구 좋았던곳이였어요...,2020-10-10,619,제주도 Jeju Island,[#빛나투어_제주✈️]


In [77]:
df.tail()

,content,date,like,place,tags
195,#아녜스의앞치마 .요즘 날씨가 완전 가을 같아요~~~^^오시는 손님마다 아 추워!!...,2021-08-16,100,아녜스의앞치마,"[#아녜스의앞치마, #제주, #제주도, #9월제주도가볼만한곳, #8월제주도가볼만한곳..."
196,..현재 인스타계정 상태를 분석한 결과와 관리하는 방법을 무료로 알려드립니다..카톡...,2021-08-15,148,,"[#감성여행, #제주도관광, #패키지여행, #정선여행, #2박3일여행, #도보여행,..."
197,제주 스위스마을2020.05.23#제주도 #jeju #제주도여행 #제주도한달살기 #...,2021-08-15,20,제주조천스위스마을,"[#제주도, #jeju, #제주도여행, #제주도한달살기, #제주한달살기, #한달살기..."
198,진짜 어떻하냐..너무 귀엽잖아!!!!,2021-08-15,65,제주 그린데이 게스트하우스 'GreenDay' jeju guesthouse & ba...,[]
199,≤붉은 방 - 붉은 색의 조화≥앙리 마티스▤ 붉은 방 ▤이 원래 파란색이였던 것알고...,2021-08-15,23,"그대, 나의 뮤즈",[]


In [81]:
df.to_csv('제주여행.csv', index=False)

In [82]:
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

In [83]:
driver.close()

## 4가지 CSV 파일의 중복을 제거한 후 통합 저장

In [89]:
for keyword in ['제주도맛집','제주맛집','제주여행']:
    df = pd.read_csv(f'{keyword}.csv')
    print(df.head())

                                             content        data  like  \
0  협찬제주썬셋맛집 "랍스터빈"주소 : 제주시 애월읍 애월해안로 752전화 : 064-...  2021-08-20     0   
1  #한라산아래첫마을 제주 서귀포시 안덕면 산록남로 675국내에선 제주가 최대 메밀 산...  2021-08-20     0   

      place                                               tags  
0       NaN  ['#애월맛집', '#애월카페', '#애월읍맛집', '#제주애월맛집', '#제주도맛...  
1  한라산아래첫마을  ['#한라산아래첫마을', '#포도냐제주도', '#제주도막국수', '#제주도여행', ...  
                                             content        data like   place  \
0  제주도 도착~~공항근처 #제주분식 에서 먹은 첫식사만족만족!#제주맛집#제주도맛집#제...  2019-01-01  705     NaN   
1  ﻿#함덕맛집 무거버거 바다 보면서 먹으니까 더 꿀맛👍🏻 #제주도카페투어#제주일상#함...  2019-01-01  384     NaN   
2  .대만족 제주스!.#동문시장#제주동문시장#제주도기념품#제주기념품#제주스는완벽#제주도...  2019-01-01  595     NaN   
3  #제주여행#2일차무거버거 제주와서 먹은것중에 제일 맛있었음!#제주도여행#제주맛집#함...  2019-01-01  412     NaN   
4  🥇1월 영업 안내🥇-1월 2일, 3일(수,목)은 쉬어갑니다-1월 4일,5일,6일(금...  2019-01-01  364  로타리과자점   

                                                tags  
0  ['#제주분식', '#제주맛집', '#제주도맛집', '#제주맛집추천